In [ ]:
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interact

from src.data_load import load_tables, load_instance, load_distances
from src.filtering import flexible_filter
from src.plotting import plot_metrics_comparison, plot_gantt_labors_by_driver, plot_results, plot_service_driver_distance, \
                         plot_gantt_by_services, plot_gantt_by_drivers
from src.metrics import collect_vt_metrics_range, collect_results_to_df, compute_metrics_with_moves, get_day_plotting_df, \
                        collect_hist_baseline_dfs
from src.experimentation_config import *
from src.config import *

data_path = '../data'
instance = 'instAS1'
# instance = 'inst3'
# instance = 'instr1'
dist_method = 'osrm'

directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
labors_real_df = load_instance(data_path, instance, labors_raw_df)
dist_dict = load_distances(data_path, dist_method, instance)

fechas = fechas_dict[instance]

# Cargar resultados

In [ ]:
labors_hist_df, moves_hist_df = collect_hist_baseline_dfs(data_path, instance, fechas)
labors_algo_df, moves_algo_df = collect_results_to_df(data_path, instance, fechas, assignment_type='algorithm')

# Results comparison

In [12]:
metricas = ['vt_count', 'num_drivers', "labores_por_conductor", 'total_distance', 
            'labor_extra_time', 'driver_extra_time', 'driver_move_distance']
# metricas = ['num_drivers', 'total_distance', 'tiempo_extra', 'driver_move_distance']

def plot_metrics_comparison_interactive(
    labors_hist_df: pd.DataFrame,
    moves_hist_df: pd.DataFrame,
    labors_algo_df: pd.DataFrame,
    moves_algo_df: pd.DataFrame,
    cities: list[str],
    metricas,
    dist_dict: dict,
    fechas: tuple[str, str]
):
    """
    Interactive version of plot_metrics_comparison with ipywidgets.
    Allows selecting the city dynamically from a dropdown.
    """

    def _plot_for_city(city: str):
        # Call your existing function
        figs = plot_metrics_comparison(
            labors_hist_df=labors_hist_df,
            moves_hist_df=moves_hist_df,
            labors_algo_df=labors_algo_df,
            moves_algo_df=moves_algo_df,
            city=city,
            metricas=metricas,
            dist_dict=dist_dict,
            fechas=fechas
        )
        # return figs

    # Create dropdown
    dropdown = widgets.Dropdown(
        options=cities,
        value=cities[0],  # default city
        description="Ciudad:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )

    # Link dropdown with plotting function
    interact(_plot_for_city, city=dropdown)

plot_metrics_comparison_interactive(
    labors_hist_df,
    moves_hist_df,
    labors_algo_df,
    moves_algo_df,
    cities=valid_cities,  # put the city codes here
    metricas=metricas,
    dist_dict=dist_dict,
    fechas=(fechas[0], fechas[-1])
)


interactive(children=(Dropdown(description='Ciudad:', layout=Layout(width='300px'), options=('149', '1', '126'…

In [9]:
def interactive_driver_distance_plot(labors_hist_df, moves_hist_df,
                                     labors_algo_df, moves_algo_df,
                                     top_n: int = 15):

    @interact(city=widgets.Dropdown(options=valid_cities, description="City:"),
              date=widgets.Dropdown(options=fechas, description="Date:"))
    def update(city, date):
        plot_service_driver_distance(labors_hist_df, moves_hist_df,
                                     labors_algo_df, moves_algo_df,
                                     city, date, top_n=top_n)

interactive_driver_distance_plot(labors_hist_df, moves_hist_df,
                             labors_algo_df, moves_algo_df,
                             top_n =15)

interactive(children=(Dropdown(description='City:', options=('149', '1', '126', '150', '844', '830', '1004'), …

# Gantt and schedules

In [18]:
day_str = '2025-09-08'

df_plot_hist, driver_col = get_day_plotting_df( labors_hist_df, day_str=day_str, city_code ='1', 
                                                dist_dict=dist_dict, assignment_type='historic')
df_plot_algo, driver_col = get_day_plotting_df( labors_algo_df, day_str=day_str, city_code ='1', 
                                                dist_dict=dist_dict, assignment_type='algorithm')

filt_labors = labors_hist_df[(labors_hist_df['schedule_date'].dt.day==8) & 
                (labors_hist_df['city']=='1')]
filt_moves = moves_hist_df[(moves_hist_df['schedule_date'].dt.day==8) & 
               (moves_hist_df['city']=='1')]

plot_gantt_labors_by_driver(df_plot_hist, day_str=day_str, driver_col='historic_driver', return_fig=False)
plot_gantt_by_drivers(filt_labors, filt_moves, day_str, TIEMPO_GRACIA, assignment_type='historic', return_fig=False)
plot_gantt_by_services(filt_labors, day_str, assignment_type='historic', return_fig=False)
# plot_results()


# plot_gantt_labors_by_driver(df_plot_algo, day_str=day_str, driver_col='assigned_driver')
# plot_results(labors_algo_df[(labors_algo_df['schedule_date'].dt.day==8) & 
#                 (labors_algo_df['city']=='149')], 
#              moves_algo_df[(moves_algo_df['schedule_date'].dt.day==8) & 
#                (moves_algo_df['city']=='149')], 
#              day_str, TIEMPO_GRACIA, assignment_type='algorithm')


⚠️ DataFrame vacío.


In [19]:
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd

# --- Wrappers around your existing plotting functions ---
def get_driver_gantt(labors_df, moves_df, day_str, city, assignment_type, tiempo_gracia):
    df_plot, driver_col = get_day_plotting_df(
        labors_df, day_str=day_str, city_code=city,
        dist_dict=dist_dict, assignment_type=assignment_type
    )
    fig = plot_gantt_labors_by_driver(
        df_plot, day_str=day_str, driver_col=driver_col, return_fig=True
    )
    return fig


def get_driver_moves_gantt(labors_df, moves_df, day_str, city, assignment_type, tiempo_gracia):
    fig = plot_gantt_by_drivers(
        labors_df[labors_df["city"] == city],
        moves_df[moves_df["city"] == city],
        day_str, tiempo_gracia, assignment_type=assignment_type, return_fig=True
    )
    return fig


def get_service_gantt(labors_df, day_str, city, assignment_type):
    fig = plot_gantt_by_services(
        labors_df[(labors_df["city"] == city) & 
                  (labors_df["schedule_date"].dt.strftime("%Y-%m-%d") == day_str)],
        day_str, assignment_type=assignment_type, return_fig=True
    )
    return fig


# --- Main dashboard ---
def plot_gantt_dashboard(
    labors_hist_df, moves_hist_df,
    labors_algo_df, moves_algo_df,
    tiempo_gracia=15, dist_dict=None
):
    # Collect available cities and dates
    cities = sorted(labors_hist_df["city"].unique().astype(str))
    all_dates = pd.to_datetime(labors_hist_df["schedule_date"].dt.date.unique())
    all_dates = sorted([d.strftime("%Y-%m-%d") for d in all_dates])

    # Dropdown widgets
    city_dd = widgets.Dropdown(options=cities, value=cities[0], description="City")
    date_dd = widgets.Dropdown(options=all_dates, value=all_dates[0], description="Date")

    out = widgets.Output()

    def update(city, day_str):
        out.clear_output(wait=True)
        with out:
            # --- Row 1: Driver Gantt ---
            fig_hist_1 = get_driver_gantt(labors_hist_df, moves_hist_df, day_str, city, "historic", tiempo_gracia)
            fig_algo_1 = get_driver_gantt(labors_algo_df, moves_algo_df, day_str, city, "algorithm", tiempo_gracia)

            # --- Row 2: Driver + Moves ---
            fig_hist_2 = get_driver_moves_gantt(labors_hist_df, moves_hist_df, day_str, city, "historic", tiempo_gracia)
            fig_algo_2 = get_driver_moves_gantt(labors_algo_df, moves_algo_df, day_str, city, "algorithm", tiempo_gracia)

            # --- Row 3: Service Gantt ---
            fig_hist_3 = get_service_gantt(labors_hist_df, day_str, city, "historic")
            fig_algo_3 = get_service_gantt(labors_algo_df, day_str, city, "algorithm")

            # Display as stacked rows with 2 columns (side-by-side)
            for f_hist, f_algo, title in [
                (fig_hist_1, fig_algo_1, "Driver Gantt"),
                (fig_hist_2, fig_algo_2, "Driver + Moves"),
                (fig_hist_3, fig_algo_3, "Service Gantt")
            ]:
                print(f"--- {title} ({day_str}, City {city}) ---")
                display(widgets.HBox([f_hist.show(), f_algo.show()]))

    # Link widgets to update function
    interact(update, city=city_dd, day_str=date_dd)
    display(out)


In [20]:
# Example usage
plot_gantt_dashboard(
    labors_hist_df, moves_hist_df,
    labors_algo_df, moves_algo_df,
    tiempo_gracia=15,   # whatever grace time you want
    dist_dict=dist_dict # your distance dictionary if needed
)


interactive(children=(Dropdown(description='City', options=(np.str_('1'), np.str_('1004'), np.str_('126'), np.…

Output()

In [21]:
import ipywidgets as widgets
from ipywidgets import interact
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_gantts_comparison(
    labors_hist_df: pd.DataFrame,
    moves_hist_df: pd.DataFrame,
    labors_algo_df: pd.DataFrame,
    moves_algo_df: pd.DataFrame,
    city: str,
    dist_dict: dict,
    day_str: str,
    tiempo_gracia: int = 15,
):
    """
    Genera gráficos comparativos (historic vs algoritmo) para 3 tipos de Gantt:
    - Labors por conductor
    - Tareas por conductor (con fallos y retrasos)
    - Labors por servicio

    Cada par de gráficos (historic, algoritmo) se muestra en una fila, side-by-side.
    """

    figs = []

    # --- 1) Gantt labors por conductor ---
    df_plot_hist, driver_col_hist = get_day_plotting_df(
        labors_hist_df, day_str=day_str, city_code=city,
        dist_dict=dist_dict, assignment_type="historic"
    )
    df_plot_algo, driver_col_algo = get_day_plotting_df(
        labors_algo_df, day_str=day_str, city_code=city,
        dist_dict=dist_dict, assignment_type="algorithm"
    )
    fig_hist = plot_gantt_labors_by_driver(df_plot_hist, day_str=day_str, driver_col=driver_col_hist, return_fig=True)
    fig_algo = plot_gantt_labors_by_driver(df_plot_algo, day_str=day_str, driver_col=driver_col_algo, return_fig=True)

    figs.append((fig_hist, fig_algo, "Labors por Conductor"))

    # --- 2) Gantt tareas por conductor (drivers+moves) ---
    filt_labors_hist = labors_hist_df[(labors_hist_df['schedule_date'].dt.date == pd.to_datetime(day_str).date()) &
                                      (labors_hist_df['city'] == city)]
    filt_moves_hist = moves_hist_df[(moves_hist_df['schedule_date'].dt.date == pd.to_datetime(day_str).date()) &
                                    (moves_hist_df['city'] == city)]
    filt_labors_algo = labors_algo_df[(labors_algo_df['schedule_date'].dt.date == pd.to_datetime(day_str).date()) &
                                      (labors_algo_df['city'] == city)]
    filt_moves_algo = moves_algo_df[(moves_algo_df['schedule_date'].dt.date == pd.to_datetime(day_str).date()) &
                                    (moves_algo_df['city'] == city)]

    fig_hist = plot_gantt_by_drivers(filt_labors_hist, filt_moves_hist, day_str, tiempo_gracia, "historic", return_fig=True)
    fig_algo = plot_gantt_by_drivers(filt_labors_algo, filt_moves_algo, day_str, tiempo_gracia, "algorithm", return_fig=True)

    figs.append((fig_hist, fig_algo, "Tareas por Conductor"))

    # --- 3) Gantt por servicios ---
    fig_hist = plot_gantt_by_services(filt_labors_hist, day_str, "historic", return_fig=True)
    fig_algo = plot_gantt_by_services(filt_labors_algo, day_str, "algorithm", return_fig=True)

    figs.append((fig_hist, fig_algo, "Labors por Servicio"))

    # --- Mostrar en filas (side-by-side) ---
    for fig_hist, fig_algo, title in figs:
        if fig_hist is None or fig_algo is None:
            continue
        print(f"=== {title} — Ciudad {city}, Día {day_str} ===")
        fig_hist.show()
        fig_algo.show()

    return figs


def plot_gantts_comparison_interactive(
    labors_hist_df: pd.DataFrame,
    moves_hist_df: pd.DataFrame,
    labors_algo_df: pd.DataFrame,
    moves_algo_df: pd.DataFrame,
    cities: list[str],
    dist_dict: dict,
    fechas: list[str],
    tiempo_gracia: int = 15,
):
    """
    Versión interactiva de plot_gantts_comparison con ipywidgets.
    Permite seleccionar ciudad y día.
    """

    def _plot(city: str, day_str: str):
        x = plot_gantts_comparison(
            labors_hist_df, moves_hist_df,
            labors_algo_df, moves_algo_df,
            city=city,
            dist_dict=dist_dict,
            day_str=day_str,
            tiempo_gracia=tiempo_gracia,
        )

    # Dropdowns
    dropdown_city = widgets.Dropdown(
        options=cities,
        value=cities[0],
        description="Ciudad:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="250px")
    )
    dropdown_date = widgets.Dropdown(
        options=fechas,
        value=fechas[0],
        description="Fecha:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="250px")
    )

    interact(_plot, city=dropdown_city, day_str=dropdown_date)


In [22]:
plot_gantts_comparison_interactive(
    labors_hist_df,
    moves_hist_df,
    labors_algo_df,
    moves_algo_df,
    cities=valid_cities,   # e.g. ["1","149",...]
    dist_dict=dist_dict,
    fechas=fechas,         # list of valid date strings
    tiempo_gracia=15
)


interactive(children=(Dropdown(description='Ciudad:', layout=Layout(width='250px'), options=('149', '1', '126'…

# Pruebas adicionales

## MISMATCH BETWEEN LABORES AND MOVES

In [8]:
moves_algo_df

print(len(labors_algo_df), len(moves_algo_df))

merged = labors_algo_df.merge(moves_algo_df.drop_duplicates(), how='left', indicator=True)

all_present = (merged["_merge"] != "left_only").all()

print("All rows of labors_algo_df are in moves_algo_df:", all_present)

missing_rows = labors_algo_df.merge(moves_algo_df.drop_duplicates(), 
                                    how='left', 
                                    indicator=True).query('_merge == "left_only"')

missing_rows


1086 2474
All rows of labors_algo_df are in moves_algo_df: False


,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,alfred,labor_start_date,labor_end_date,...,actual_start,actual_end,dist_km,date,n_drivers,start_point,end_point,distance_km,duration_min,_merge
398,251440,347314.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-06 09:30:00-05:00,NaN,71852.0,2026-01-06 08:51:00-05:00,2026-01-06 09:59:00-05:00,...,NaT,NaT,0.0,2026-01-06,1,NaN,NaN,NaN,NaN,left_only
399,252159,348067.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-06 12:00:00-05:00,NaN,14946.0,2026-01-06 10:10:57-05:00,2026-01-06 10:49:18-05:00,...,NaT,NaT,0.0,2026-01-06,1,NaN,NaN,NaN,NaN,left_only
400,252159,348068.0,3.0,Wash and Polish,WASH_AND_POLISH,2026-01-06 12:00:00-05:00,4071.0,NaN,2026-01-06 10:49:18-05:00,2026-01-06 11:59:26-05:00,...,NaT,NaT,0.0,2026-01-06,1,NaN,NaN,NaN,NaN,left_only
401,252159,348069.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,2026-01-06 12:00:00-05:00,NaN,14946.0,2026-01-06 11:59:26-05:00,2026-01-06 12:26:44-05:00,...,NaT,NaT,0.0,2026-01-06,1,NaN,NaN,NaN,NaN,left_only
405,252494,348423.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-06 11:00:00-05:00,NaN,71852.0,2026-01-06 11:30:00-05:00,2026-01-06 12:27:00-05:00,...,NaT,NaT,0.0,2026-01-06,1,NaN,NaN,NaN,NaN,left_only
406,252497,348427.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-06 12:30:00-05:00,NaN,71852.0,2026-01-06 12:39:00-05:00,2026-01-06 14:02:00-05:00,...,NaT,NaT,0.0,2026-01-06,1,NaN,NaN,NaN,NaN,left_only
407,252607,348555.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-06 15:00:00-05:00,NaN,14946.0,2026-01-06 15:01:00-05:00,2026-01-06 15:30:00-05:00,...,NaT,NaT,0.0,2026-01-06,1,NaN,NaN,NaN,NaN,left_only
408,252675,348628.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-06 16:00:00-05:00,NaN,71852.0,2026-01-06 15:59:00-05:00,2026-01-06 17:29:00-05:00,...,NaT,NaT,0.0,2026-01-06,1,NaN,NaN,NaN,NaN,left_only
416,227886,322568.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 12:00:00-05:00,NaN,71852.0,2026-01-08 17:44:00-05:00,2026-01-08 19:04:00-05:00,...,NaT,NaT,0.0,2026-01-08,1,NaN,NaN,NaN,NaN,left_only
425,262156,358770.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-09 10:00:00-05:00,NaN,14946.0,2026-01-09 09:12:00-05:00,2026-01-09 10:31:00-05:00,...,NaT,NaT,0.0,2026-01-09,1,NaN,NaN,NaN,NaN,left_only


In [11]:
flexible_filter(labors_algo_df,
                service_id=251440)

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,alfred,labor_start_date,labor_end_date,...,city,state_service,map_start_point,map_end_point,assigned_driver,actual_start,actual_end,dist_km,date,n_drivers
247,251440,347314.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-06 09:30:00-05:00,NaN,71852.0,2026-01-06 08:51:00-05:00,2026-01-06 09:59:00-05:00,...,126,COMPLETED,POINT (-74.8252164 11.0197123),POINT (-74.80465445403671 11.003221388928484),<NA>,NaT,NaT,0.0,2026-01-06,1


## Whatever

### Labor adicional en Sep 8 en Bogotá / Mismatch en número de labores de VT

In [23]:
# Número de labores de transporte en el registro real
num_labores = len(flexible_filter(
    labors_real_df,
    city="149",
    schedule_date=lambda s: s.dt.day == 8,
    labor_category="VEHICLE_TRANSPORTATION",
))
print(f'Número de labores: {num_labores}')

Número de labores: 76


In [24]:
# Servicios con asignación real que no se pudieron cumplir por 
flexible_filter(
    labors_hist_df,
    city="149",
    schedule_date=lambda s: s.dt.day == 8,
    labor_category="VEHICLE_TRANSPORTATION",
    historic_driver='na'
)

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,historic_driver,historic_start,historic_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers
492,253255,349260.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 08:30:00-05:00,NaN,<NA>,NaT,NaT,...,139870.0,POINT (-74.0488999 4.714100499999999),10.0,POINT (-74.075015 4.69416),149,COMPLETED,POINT (-74.0488999 4.714100499999999),POINT (-74.075015 4.69416),2026-01-08,20
506,254242,350916.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 13:00:00-05:00,NaN,<NA>,NaT,NaT,...,110940.0,POINT (-74.1306479 4.6875541),130938.0,POINT (-74.1362453 4.6927107),149,COMPLETED,POINT (-74.1306479 4.6875541),POINT (-74.1362453 4.6927107),2026-01-08,20
555,255033,351199.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,2026-01-08 17:50:00-05:00,NaN,<NA>,NaT,NaT,...,78914.0,POINT (-74.0459602 4.7526023),78914.0,POINT (-74.0459602 4.7526023),149,COMPLETED,POINT (-74.0459602 4.7526023),POINT (-74.0459602 4.7526023),2026-01-08,20


In [25]:
# Servicios con asignación del algoritmo que no se pudieron cumplir
flexible_filter(
    labors_algo_df,
    city="149",
    schedule_date=lambda s: s.dt.day == 8,
    labor_category="VEHICLE_TRANSPORTATION",
    actual_start='na'
)

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,assigned_driver,actual_start,actual_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers


In [26]:
flexible_filter(
    labors_hist_df,
    city="149",
    schedule_date=lambda s: s.dt.day == 8,
    alfred=35582
).sort_values(by=['schedule_date'])

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,historic_driver,historic_start,historic_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers


In [27]:
flexible_filter(
    labors_raw_df,
    city="149",
    # schedule_date=lambda s: s.dt.day == 8,
    alfred=35582,
    service_id=[211604, 211612, 211608, 211599, 211611]
).sort_values(by=['schedule_date'])

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,alfred,labor_start_date,labor_end_date,address_id,address_point,address_name,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service
87058,211604,304074.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-02-12 10:00:00-05:00,NaN,35582.0,2025-02-12 10:00:00-05:00,2025-02-12 10:00:00-05:00,55386.0,POINT (-74.0844326 4.7205401),Casa,113943.0,POINT (-74.0299644 4.8484015),113944.0,POINT (-74.07620949999999 4.6931322),149,COMPLETED
87064,211612,304082.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-02-12 10:30:00-05:00,NaN,35582.0,2025-02-12 10:30:00-05:00,2025-02-12 10:30:00-05:00,55386.0,POINT (-74.0844326 4.7205401),Casa,113943.0,POINT (-74.0299644 4.8484015),113944.0,POINT (-74.07620949999999 4.6931322),149,COMPLETED
87061,211608,304078.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-02-12 11:30:00-05:00,NaN,35582.0,2025-02-12 11:30:00-05:00,2025-02-12 11:30:00-05:00,55386.0,POINT (-74.0844326 4.7205401),Casa,113943.0,POINT (-74.0299644 4.8484015),113944.0,POINT (-74.07620949999999 4.6931322),149,COMPLETED
87055,211599,304069.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-02-12 12:00:00-05:00,NaN,35582.0,2025-02-12 12:00:00-05:00,2025-02-12 12:00:00-05:00,55386.0,POINT (-74.0844326 4.7205401),Casa,113943.0,POINT (-74.0299644 4.8484015),113944.0,POINT (-74.07620949999999 4.6931322),149,COMPLETED
87063,211611,304081.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-02-12 12:00:00-05:00,NaN,35582.0,2025-02-12 12:00:00-05:00,2025-02-12 12:00:00-05:00,55386.0,POINT (-74.0844326 4.7205401),Casa,113943.0,POINT (-74.0299644 4.8484015),113944.0,POINT (-74.07620949999999 4.6931322),149,COMPLETED


In [28]:
flexible_filter(
    labors_hist_df,
    service_id=211608
)

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,historic_driver,historic_start,historic_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers
40,211608,304078.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2026-01-05 11:30:00-05:00,NaN,35582.0,2026-01-05 11:29:32.892000-05:00,2026-01-05 12:37:55.560000-05:00,...,113943.0,POINT (-74.0299644 4.8484015),113944.0,POINT (-74.07620949999999 4.6931322),149,COMPLETED,POINT (-74.0299644 4.8484015),POINT (-74.07620949999999 4.6931322),2026-01-05,19


In [29]:
# Number of transport services computed by compute_metrics_with_moves
metrics_algo_df = compute_metrics_with_moves(labors_algo_df, moves_algo_df, fechas, dist_dict, 
                                                 8, '149', 'algorithm', False, 'haversine')
metrics_algo_df

,day,vt_count,num_drivers,labores_por_conductor,utilizacion_promedio_%,labor_extra_time,driver_extra_time,total_distance,driver_move_distance
0,2026-01-05,72,16,4.500,51.8,943.0,1299.3,0.0,495.2949
1,2026-01-06,70,15,4.667,45.4,1124.4,1519.2,0.0,224.0323
2,2026-01-07,79,19,4.158,40.8,1259.1,1917.0,0.0,401.5647
3,2026-01-08,76,15,5.067,47.5,1177.7,1577.7,0.0,392.8093
4,2026-01-09,79,19,4.158,39.8,1103.3,1562.5,0.0,357.1168
5,2026-01-10,67,13,5.154,49.2,1067.0,1333.3,0.0,348.5280
6,2026-01-11,69,14,4.929,50.4,1123.2,1508.7,0.0,282.5524


### Diferencia driver_move_distance, servicio 207215 (city=1)

In [30]:
service_id = 207215

In [31]:
# Labor real 
flexible_filter(
    labors_real_df,
    service_id=service_id
)

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,alfred,labor_start_date,labor_end_date,address_id,address_point,address_name,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service
180,207215,299281.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-10 17:00:00-05:00,NaN,2036.0,2026-01-10 16:45:00-05:00,2026-01-10 17:39:00-05:00,55748.0,POINT (-75.55748439999999 6.252939599999999),Apartamento,4852.0,POINT (-75.56991479999999 6.2276503),47747.0,POINT (-75.55834349999999 6.2087283),1,COMPLETED


In [32]:
# Movimientos de la reconstrucción de Alfred
flexible_filter(
    moves_hist_df,
    service_id=service_id
)

,service_id,labor_id,labor_name,labor_category,historic_driver,schedule_date,historic_start,historic_end,start_point,end_point,distance_km,duration_min,city,date
2308,207215,299281,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2036.0,2026-01-10 17:00:00-05:00,2026-01-10 16:30:00-05:00,2026-01-10 17:05:20.427000-05:00,POINT (-75.56991479999999 6.2276503),POINT (-75.55834349999999 6.2087283),NaN,35.3,1,2026-01-10
2306,207215,299281_free,FREE_TIME,FREE_TIME,2036.0,2026-01-10 17:00:00-05:00,2026-01-10 14:05:20.427000-05:00,2026-01-10 16:30:00-05:00,POINT (-75.56991479999999 6.2276503),POINT (-75.56991479999999 6.2276503),0.0,144.7,1,2026-01-10
2307,207215,299281_move,DRIVER_MOVE,DRIVER_MOVE,2036.0,2026-01-10 17:00:00-05:00,2026-01-10 16:30:00-05:00,2026-01-10 16:30:00-05:00,POINT (-75.56991479999999 6.2276503),POINT (-75.56991479999999 6.2276503),0.0,0.0,1,2026-01-10


In [33]:
# Movimientos del algoritmo
flexible_filter(
    moves_algo_df,
    service_id=service_id
)

,service_id,labor_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
2138,207215,299281,Alfred Initial Transport,VEHICLE_TRANSPORTATION,Luis Fernando Matabanchoy,2026-01-10 17:00:00-05:00,2026-01-10 16:30:00-05:00,2026-01-10 17:05:20.427000-05:00,POINT (-75.56991479999999 6.2276503),POINT (-75.55834349999999 6.2087283),NaN,35.3,1,2026-01-10
2136,207215,299281_free,FREE_TIME,FREE_TIME,Luis Fernando Matabanchoy,2026-01-10 17:00:00-05:00,2026-01-10 13:13:52.428000-05:00,2026-01-10 16:22:08.688000-05:00,POINT (-75.572374 6.2506087),POINT (-75.572374 6.2506087),0.0000,188.3,1,2026-01-10
2137,207215,299281_move,DRIVER_MOVE,DRIVER_MOVE,Luis Fernando Matabanchoy,2026-01-10 17:00:00-05:00,2026-01-10 16:22:08.688000-05:00,2026-01-10 16:30:00-05:00,POINT (-75.572374 6.2506087),POINT (-75.56991479999999 6.2276503),3.9276,7.9,1,2026-01-10


In [34]:
# Labores del conductor de la reconstrucción de Alfred
flexible_filter(
    labors_hist_df,
    historic_driver = 2036, 
    schedule_date=lambda s: s.dt.day == 8,
).sort_values(by=['schedule_date'])

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,historic_driver,historic_start,historic_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers
560,226655,320621.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 08:00:00-05:00,NaN,2036.0,2026-01-08 07:30:00-05:00,2026-01-08 08:03:46.134000-05:00,...,111980.0,POINT (-75.5724749 6.2026219),102162.0,POINT (-75.564082 6.2082589),1,COMPLETED,POINT (-75.5724749 6.2026219),POINT (-75.564082 6.2082589),2026-01-08,7
568,227462,321491.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 10:00:00-05:00,NaN,2036.0,2026-01-08 09:30:00-05:00,2026-01-08 10:04:13.611000-05:00,...,130686.0,POINT (-75.5733585 6.1975044),117281.0,POINT (-75.56979539999999 6.2198631),1,COMPLETED,POINT (-75.5733585 6.1975044),POINT (-75.56979539999999 6.2198631),2026-01-08,7
557,225964,319872.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 13:00:00-05:00,NaN,2036.0,2026-01-08 12:30:00-05:00,2026-01-08 13:10:04.044000-05:00,...,130351.0,POINT (-75.56342339999999 6.1909383),4852.0,POINT (-75.56991479999999 6.2276503),1,COMPLETED,POINT (-75.56342339999999 6.1909383),POINT (-75.56991479999999 6.2276503),2026-01-08,7
583,227914,321978.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 15:30:00-05:00,NaN,2036.0,2026-01-08 15:00:00-05:00,2026-01-08 15:34:33.951000-05:00,...,117281.0,POINT (-75.56979539999999 6.2198631),130810.0,POINT (-75.5763979 6.203141999999999),1,COMPLETED,POINT (-75.56979539999999 6.2198631),POINT (-75.5763979 6.203141999999999),2026-01-08,7
585,227973,322046.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 17:00:00-05:00,NaN,2036.0,2026-01-08 16:30:00-05:00,2026-01-08 17:10:04.044000-05:00,...,4852.0,POINT (-75.56991479999999 6.2276503),130854.0,POINT (-75.56342339999999 6.1909383),1,COMPLETED,POINT (-75.56991479999999 6.2276503),POINT (-75.56342339999999 6.1909383),2026-01-08,7


In [35]:
# Labores del conductor de la reconstrucción de Alfred
flexible_filter(
    moves_algo_df,
    service_id=[207253,service_id]
).sort_values(by=['schedule_date'])

,service_id,labor_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
2126,207253,299329,Alfred Initial Transport,VEHICLE_TRANSPORTATION,Jaime Enrique Dueñas,2026-01-10 14:00:00-05:00,2026-01-10 13:30:00-05:00,2026-01-10 14:05:20.427000-05:00,POINT (-75.55834349999999 6.2087283),POINT (-75.56991479999999 6.2276503),NaN,35.3,1,2026-01-10
2124,207253,299329_free,FREE_TIME,FREE_TIME,Jaime Enrique Dueñas,2026-01-10 14:00:00-05:00,2026-01-10 10:10:04.341000-05:00,2026-01-10 13:28:19.164000-05:00,POINT (-75.564082 6.2082589),POINT (-75.564082 6.2082589),0.0000,198.2,1,2026-01-10
2125,207253,299329_move,DRIVER_MOVE,DRIVER_MOVE,Jaime Enrique Dueñas,2026-01-10 14:00:00-05:00,2026-01-10 13:28:19.164000-05:00,2026-01-10 13:30:00-05:00,POINT (-75.564082 6.2082589),POINT (-75.55834349999999 6.2087283),0.8403,1.7,1,2026-01-10
2138,207215,299281,Alfred Initial Transport,VEHICLE_TRANSPORTATION,Luis Fernando Matabanchoy,2026-01-10 17:00:00-05:00,2026-01-10 16:30:00-05:00,2026-01-10 17:05:20.427000-05:00,POINT (-75.56991479999999 6.2276503),POINT (-75.55834349999999 6.2087283),NaN,35.3,1,2026-01-10
2136,207215,299281_free,FREE_TIME,FREE_TIME,Luis Fernando Matabanchoy,2026-01-10 17:00:00-05:00,2026-01-10 13:13:52.428000-05:00,2026-01-10 16:22:08.688000-05:00,POINT (-75.572374 6.2506087),POINT (-75.572374 6.2506087),0.0000,188.3,1,2026-01-10
2137,207215,299281_move,DRIVER_MOVE,DRIVER_MOVE,Luis Fernando Matabanchoy,2026-01-10 17:00:00-05:00,2026-01-10 16:22:08.688000-05:00,2026-01-10 16:30:00-05:00,POINT (-75.572374 6.2506087),POINT (-75.56991479999999 6.2276503),3.9276,7.9,1,2026-01-10


In [36]:
# Labores del conductor de la reconstrucción de Alfred
flexible_filter(
    labors_algo_df,
    assigned_driver = 'Jaime Yepes', 
    schedule_date=lambda s: s.dt.day == 8,
).sort_values(by=['schedule_date'])

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,assigned_driver,actual_start,actual_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers
584,227929,321995.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 15:30:00-05:00,NaN,Jaime Yepes,2026-01-08 15:00:00-05:00,2026-01-08 15:44:23.730000-05:00,...,16344.0,POINT (-75.6091325 6.157386300000001),130471.0,POINT (-75.5740513 6.219988099999999),1,COMPLETED,POINT (-75.6091325 6.157386300000001),POINT (-75.5740513 6.219988099999999),2026-01-08,9
559,226535,320474.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 16:30:00-05:00,NaN,Jaime Yepes,2026-01-08 16:00:00-05:00,2026-01-08 16:38:40.344000-05:00,...,4852.0,POINT (-75.56991479999999 6.2276503),130438.0,POINT (-75.5520973 6.202223299999999),1,COMPLETED,POINT (-75.56991479999999 6.2276503),POINT (-75.5520973 6.202223299999999),2026-01-08,9


In [37]:
flexible_filter(
    labors_hist_df,
    historic_driver = 2036, 
    schedule_date=lambda s: s.dt.day == 8,
).sort_values(by=['schedule_date'])

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,historic_driver,historic_start,historic_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers
560,226655,320621.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 08:00:00-05:00,NaN,2036.0,2026-01-08 07:30:00-05:00,2026-01-08 08:03:46.134000-05:00,...,111980.0,POINT (-75.5724749 6.2026219),102162.0,POINT (-75.564082 6.2082589),1,COMPLETED,POINT (-75.5724749 6.2026219),POINT (-75.564082 6.2082589),2026-01-08,7
568,227462,321491.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 10:00:00-05:00,NaN,2036.0,2026-01-08 09:30:00-05:00,2026-01-08 10:04:13.611000-05:00,...,130686.0,POINT (-75.5733585 6.1975044),117281.0,POINT (-75.56979539999999 6.2198631),1,COMPLETED,POINT (-75.5733585 6.1975044),POINT (-75.56979539999999 6.2198631),2026-01-08,7
557,225964,319872.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 13:00:00-05:00,NaN,2036.0,2026-01-08 12:30:00-05:00,2026-01-08 13:10:04.044000-05:00,...,130351.0,POINT (-75.56342339999999 6.1909383),4852.0,POINT (-75.56991479999999 6.2276503),1,COMPLETED,POINT (-75.56342339999999 6.1909383),POINT (-75.56991479999999 6.2276503),2026-01-08,7
583,227914,321978.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 15:30:00-05:00,NaN,2036.0,2026-01-08 15:00:00-05:00,2026-01-08 15:34:33.951000-05:00,...,117281.0,POINT (-75.56979539999999 6.2198631),130810.0,POINT (-75.5763979 6.203141999999999),1,COMPLETED,POINT (-75.56979539999999 6.2198631),POINT (-75.5763979 6.203141999999999),2026-01-08,7
585,227973,322046.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 17:00:00-05:00,NaN,2036.0,2026-01-08 16:30:00-05:00,2026-01-08 17:10:04.044000-05:00,...,4852.0,POINT (-75.56991479999999 6.2276503),130854.0,POINT (-75.56342339999999 6.1909383),1,COMPLETED,POINT (-75.56991479999999 6.2276503),POINT (-75.56342339999999 6.1909383),2026-01-08,7


### Diferencia driver_move_distance, servicio 211424 (city=149)

In [38]:
flexible_filter(
    labors_real_df,
    service_id=211424
)

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,alfred,labor_start_date,labor_end_date,address_id,address_point,address_name,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service
831,211424,303876.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-05 09:00:00-05:00,NaN,211.0,2026-01-05 08:21:00-05:00,2026-01-05 11:03:00-05:00,56465.0,POINT (-74.0877144 4.7271318),Casa,113944.0,POINT (-74.07620949999999 4.6931322),113943.0,POINT (-74.0299644 4.8484015),149,COMPLETED
832,211424,304380.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,2026-01-05 09:00:00-05:00,NaN,211.0,2026-01-05 11:00:00-05:00,2026-01-05 12:07:00-05:00,56465.0,POINT (-74.0877144 4.7271318),Casa,113944.0,POINT (-74.07620949999999 4.6931322),113943.0,POINT (-74.0299644 4.8484015),149,COMPLETED
833,211424,304379.0,16.0,Tanqueo Terpel (Corriente),OTHER_LABORS,2026-01-05 09:00:00-05:00,341.0,NaN,2026-01-05 11:03:00-05:00,2026-01-05 11:03:00-05:00,11062.0,POINT (-74.04343664646149 4.750731262016588),Terpel Calle 170,113944.0,POINT (-74.07620949999999 4.6931322),113943.0,POINT (-74.0299644 4.8484015),149,COMPLETED


In [39]:
flexible_filter(
    moves_hist_df,
    service_id=211424
)

,service_id,labor_id,labor_name,labor_category,historic_driver,schedule_date,historic_start,historic_end,start_point,end_point,distance_km,duration_min,city,date
46,211424,303876,Alfred Initial Transport,VEHICLE_TRANSPORTATION,211.0,2026-01-05 09:00:00-05:00,2026-01-05 08:30:00-05:00,2026-01-05 09:08:22.299000-05:00,POINT (-74.07620949999999 4.6931322),POINT (-74.0877144 4.7271318),NaN,38.4,149,2026-01-05
50,211424,304379,Tanqueo Terpel (Corriente),OTHER_LABORS,<NA>,2026-01-05 09:00:00-05:00,2026-01-05 09:08:22.299000-05:00,2026-01-05 09:12:33.477571-05:00,POINT (-74.04343664646149 4.750731262016588),POINT (-74.04343664646149 4.750731262016588),NaN,4.2,149,2026-01-05
53,211424,304380,Alfred Transport,VEHICLE_TRANSPORTATION,211.0,2026-01-05 09:00:00-05:00,2026-01-05 09:12:33.477571-05:00,2026-01-05 09:42:33.477571-05:00,POINT (-74.0877144 4.7271318),POINT (-74.0877144 4.7271318),NaN,30.0,149,2026-01-05
44,211424,303876_free,FREE_TIME,FREE_TIME,211.0,2026-01-05 09:00:00-05:00,2026-01-05 07:06:01.683000-05:00,2026-01-05 08:29:37.356000-05:00,POINT (-74.075015 4.69416),POINT (-74.075015 4.69416),0.0000,83.6,149,2026-01-05
45,211424,303876_move,DRIVER_MOVE,DRIVER_MOVE,211.0,2026-01-05 09:00:00-05:00,2026-01-05 08:29:37.356000-05:00,2026-01-05 08:30:00-05:00,POINT (-74.075015 4.69416),POINT (-74.07620949999999 4.6931322),0.1887,0.4,149,2026-01-05
51,211424,304380_free,FREE_TIME,FREE_TIME,211.0,2026-01-05 09:00:00-05:00,2026-01-05 09:08:22.299000-05:00,2026-01-05 09:12:33.477571-05:00,POINT (-74.0877144 4.7271318),POINT (-74.0877144 4.7271318),0.0000,4.2,149,2026-01-05
52,211424,304380_move,DRIVER_MOVE,DRIVER_MOVE,211.0,2026-01-05 09:00:00-05:00,2026-01-05 09:12:33.477571-05:00,2026-01-05 09:12:33.477571-05:00,POINT (-74.0877144 4.7271318),POINT (-74.0877144 4.7271318),0.0000,0.0,149,2026-01-05


In [40]:
flexible_filter(
    moves_algo_df,
    service_id=211424
)

,service_id,labor_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
35,211424,303876,Alfred Initial Transport,VEHICLE_TRANSPORTATION,Ancisar Echavarría,2026-01-05 09:00:00-05:00,2026-01-05 08:59:53.145000-05:00,2026-01-05 09:38:15.444000-05:00,POINT (-74.07620949999999 4.6931322),POINT (-74.0877144 4.7271318),NaN,38.4,149,2026-01-05
59,211424,304379,Tanqueo Terpel (Corriente),OTHER_LABORS,<NA>,2026-01-05 09:00:00-05:00,2026-01-05 09:38:15.444000-05:00,2026-01-05 09:42:26.622571-05:00,POINT (-74.04343664646149 4.750731262016588),POINT (-74.04343664646149 4.750731262016588),NaN,4.2,149,2026-01-05
62,211424,304380,Alfred Transport,VEHICLE_TRANSPORTATION,Ancisar Echavarría,2026-01-05 09:00:00-05:00,2026-01-05 09:42:26.622571-05:00,2026-01-05 10:12:26.622571-05:00,POINT (-74.0877144 4.7271318),POINT (-74.0877144 4.7271318),NaN,30.0,149,2026-01-05
34,211424,303876_move,DRIVER_MOVE,DRIVER_MOVE,Ancisar Echavarría,2026-01-05 09:00:00-05:00,2026-01-05 08:51:41.169000-05:00,2026-01-05 08:59:53.145000-05:00,POINT (-74.08921640000001 4.698600600000001),POINT (-74.07620949999999 4.6931322),4.0998,8.2,149,2026-01-05
60,211424,304380_free,FREE_TIME,FREE_TIME,Ancisar Echavarría,2026-01-05 09:00:00-05:00,2026-01-05 09:38:15.444000-05:00,2026-01-05 09:42:26.622571-05:00,POINT (-74.0877144 4.7271318),POINT (-74.0877144 4.7271318),0.0000,4.2,149,2026-01-05
61,211424,304380_move,DRIVER_MOVE,DRIVER_MOVE,Ancisar Echavarría,2026-01-05 09:00:00-05:00,2026-01-05 09:42:26.622571-05:00,2026-01-05 09:42:26.622571-05:00,POINT (-74.0877144 4.7271318),POINT (-74.0877144 4.7271318),0.0000,0.0,149,2026-01-05


In [41]:
flexible_filter(
    moves_algo_df,
    service_id=211606
)

,service_id,labor_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
65,211606,304076,Alfred Initial Transport,VEHICLE_TRANSPORTATION,Julio Cesar Parra,2026-01-05 10:30:00-05:00,2026-01-05 10:00:00-05:00,2026-01-05 11:08:22.668000-05:00,POINT (-74.0299644 4.8484015),POINT (-74.07620949999999 4.6931322),NaN,68.4,149,2026-01-05
63,211606,304076_free,FREE_TIME,FREE_TIME,Julio Cesar Parra,2026-01-05 10:30:00-05:00,2026-01-05 08:33:25.521000-05:00,2026-01-05 09:29:03.624000-05:00,POINT (-74.023738 4.7282445),POINT (-74.023738 4.7282445),0.0000,55.6,149,2026-01-05
64,211606,304076_move,DRIVER_MOVE,DRIVER_MOVE,Julio Cesar Parra,2026-01-05 10:30:00-05:00,2026-01-05 09:29:03.624000-05:00,2026-01-05 10:00:00-05:00,POINT (-74.023738 4.7282445),POINT (-74.0299644 4.8484015),15.4698,30.9,149,2026-01-05


In [42]:
flexible_filter(
    labors_algo_df,
    assigned_driver = 'Cesar Augusto Enciso', 
    schedule_date=lambda s: s.dt.day == 8,
).sort_values(by=['schedule_date'])

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,assigned_driver,actual_start,actual_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers
482,252092,347998.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2026-01-08 13:30:00-05:00,NaN,Cesar Augusto Enciso,2026-01-08 13:00:00-05:00,2026-01-08 13:35:40.092000-05:00,...,120239.0,POINT (-74.15845829999999 4.6152668),120239.0,POINT (-74.15845829999999 4.6152668),149,COMPLETED,POINT (-74.15845829999999 4.6152668),POINT (-74.1378628 4.6258655),2026-01-08,20
484,252092,348000.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,2026-01-08 13:30:00-05:00,NaN,Cesar Augusto Enciso,2026-01-08 14:52:09.592000-05:00,2026-01-08 15:27:28.669000-05:00,...,120239.0,POINT (-74.15845829999999 4.6152668),120239.0,POINT (-74.15845829999999 4.6152668),149,COMPLETED,POINT (-74.1378628 4.6258655),POINT (-74.1474839 4.6118768),2026-01-08,20


### Diferencia driver_move_distance, servicio 207150 (city=1)

In [43]:
directorio_hist_df = labors_raw_df[labors_raw_df['city'].isin(valid_cities)][['alfred', 'address_id','address_point']].drop_duplicates().dropna(subset=['alfred', 'address_id'])
directorio_hist_df.to_csv(f'{data_path}/data_pre/directorio_hist_df.csv', index=False)
directorio_hist_df

,alfred,address_id,address_point
0,2306.0,109102.0,POINT (-76.5710842 3.406073)
3,14662.0,23056.0,POINT (-74.0314483 4.7605335)
5,6229.0,11727.0,POINT (-74.0583698 4.6510839)
6,9480.0,14935.0,POINT (-75.5838536 6.2255503)
7,12077.0,19281.0,POINT (-76.5468388 3.4090961)
...,...,...,...
92288,71852.0,130778.0,POINT (-74.7938234 10.9313923)
93318,72164.0,131621.0,POINT (-75.7102592 4.801893499999999)
93912,21405.0,28134.0,POINT (-75.6004631859725 6.254282787660095)
94113,72680.0,132403.0,POINT (-76.4651154 3.4282175)


In [44]:
flexible_filter(
    labors_hist_df,
    schedule_date=lambda s: s.dt.day == 8,
    alfred=21407
)

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,historic_driver,historic_start,historic_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers


In [45]:
flexible_filter(
    moves_hist_df,
    service_id=207150
)

,service_id,labor_id,labor_name,labor_category,historic_driver,schedule_date,historic_start,historic_end,start_point,end_point,distance_km,duration_min,city,date
2257,207150,299198,Alfred Initial Transport,VEHICLE_TRANSPORTATION,21407.0,2026-01-10 09:30:00-05:00,2026-01-10 09:00:00-05:00,2026-01-10 09:50:30.228000-05:00,POINT (-75.5506912 6.156381499999999),POINT (-75.57438254356384 6.2283712162642235),NaN,50.5,1,2026-01-10
2255,207150,299198_free,FREE_TIME,FREE_TIME,21407.0,2026-01-10 09:30:00-05:00,2026-01-10 07:00:00-05:00,2026-01-10 08:21:48.408000-05:00,POINT (-75.61853889999999 6.1358762),POINT (-75.61853889999999 6.1358762),0.0000,81.8,1,2026-01-10
2256,207150,299198_move,DRIVER_MOVE,DRIVER_MOVE,21407.0,2026-01-10 09:30:00-05:00,2026-01-10 08:21:48.408000-05:00,2026-01-10 09:00:00-05:00,POINT (-75.61853889999999 6.1358762),POINT (-75.5506912 6.156381499999999),19.0966,38.2,1,2026-01-10


In [46]:
flexible_filter(
    moves_algo_df,
    service_id=207150
)

,service_id,labor_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
2091,207150,299198,Alfred Initial Transport,VEHICLE_TRANSPORTATION,José Gonzáles Parra,2026-01-10 09:30:00-05:00,2026-01-10 09:00:00-05:00,2026-01-10 09:50:30.228000-05:00,POINT (-75.5506912 6.156381499999999),POINT (-75.57438254356384 6.2283712162642235),NaN,50.5,1,2026-01-10
2089,207150,299198_free,FREE_TIME,FREE_TIME,José Gonzáles Parra,2026-01-10 09:30:00-05:00,2026-01-10 08:35:57.093000-05:00,2026-01-10 08:39:57.528000-05:00,POINT (-75.564082 6.2082589),POINT (-75.564082 6.2082589),0.0000,4.0,1,2026-01-10
2090,207150,299198_move,DRIVER_MOVE,DRIVER_MOVE,José Gonzáles Parra,2026-01-10 09:30:00-05:00,2026-01-10 08:39:57.528000-05:00,2026-01-10 09:00:00-05:00,POINT (-75.564082 6.2082589),POINT (-75.5506912 6.156381499999999),10.0206,20.0,1,2026-01-10


### Diferencia driver_move_distance, servicio 211424 (city=149)